In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flight-price-prediction/Data_Train.csv
/kaggle/input/flight-price-prediction/Test_set.csv


In [2]:
train_data=pd.read_csv('/kaggle/input/flight-price-prediction/Data_Train.csv',index_col='Airline')
test_data=pd.read_csv('/kaggle/input/flight-price-prediction/Test_set.csv' ,index_col='Airline')

In [3]:
train_data.head()

,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
Airline,,,,,,,,,,
IndiGo,24/03/2019,Banglore,New Delhi,BLR ? DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
Air India,1/05/2019,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662
Jet Airways,9/06/2019,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
IndiGo,12/05/2019,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218
IndiGo,01/03/2019,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10683 entries, IndiGo to Air India
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date_of_Journey  10683 non-null  object
 1   Source           10683 non-null  object
 2   Destination      10683 non-null  object
 3   Route            10682 non-null  object
 4   Dep_Time         10683 non-null  object
 5   Arrival_Time     10683 non-null  object
 6   Duration         10683 non-null  object
 7   Total_Stops      10682 non-null  object
 8   Additional_Info  10683 non-null  object
 9   Price            10683 non-null  int64 
dtypes: int64(1), object(9)
memory usage: 918.1+ KB


In [5]:
# Convert with correct format (day/month/year)
train_data['Date_of_Journey'] = pd.to_datetime(train_data['Date_of_Journey'], format='%d/%m/%Y')

# Extract day and month
train_data['Day_of_Journey'] = train_data['Date_of_Journey'].dt.day
train_data['Month_of_Journey'] = train_data['Date_of_Journey'].dt.month
train_data['Year_of_Journey'] = train_data['Date_of_Journey'].dt.year


In [6]:
train_data['Day_of_Journey']=train_data['Day_of_Journey'].astype(int)
train_data['Month_of_Journey']=train_data['Month_of_Journey'].astype(int)
train_data['Year_of_Journey']=train_data['Year_of_Journey'].astype(int)

In [7]:
train_data.head()

,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Day_of_Journey,Month_of_Journey,Year_of_Journey
Airline,,,,,,,,,,,,,
IndiGo,2019-03-24,Banglore,New Delhi,BLR ? DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3,2019
Air India,2019-05-01,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662,1,5,2019
Jet Airways,2019-06-09,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,9,6,2019
IndiGo,2019-05-12,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218,12,5,2019
IndiGo,2019-03-01,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302,1,3,2019


In [8]:
train_data['Arrival_Time']=train_data['Arrival_Time'].apply(lambda x: x.split(" ")[0])

In [9]:
train_data['Arrival_hour']=train_data['Arrival_Time'].str.split(':').str[0]
train_data['Arrival_min']=train_data['Arrival_Time'].str.split(':').str[1]

In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10683 entries, IndiGo to Air India
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date_of_Journey   10683 non-null  datetime64[ns]
 1   Source            10683 non-null  object        
 2   Destination       10683 non-null  object        
 3   Route             10682 non-null  object        
 4   Dep_Time          10683 non-null  object        
 5   Arrival_Time      10683 non-null  object        
 6   Duration          10683 non-null  object        
 7   Total_Stops       10682 non-null  object        
 8   Additional_Info   10683 non-null  object        
 9   Price             10683 non-null  int64         
 10  Day_of_Journey    10683 non-null  int64         
 11  Month_of_Journey  10683 non-null  int64         
 12  Year_of_Journey   10683 non-null  int64         
 13  Arrival_hour      10683 non-null  object        
 14  Arrival_min       

In [11]:
train_data.head()

,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Day_of_Journey,Month_of_Journey,Year_of_Journey,Arrival_hour,Arrival_min
Airline,,,,,,,,,,,,,,,
IndiGo,2019-03-24,Banglore,New Delhi,BLR ? DEL,22:20,01:10,2h 50m,non-stop,No info,3897,24,3,2019,01,10
Air India,2019-05-01,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662,1,5,2019,13,15
Jet Airways,2019-06-09,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25,19h,2 stops,No info,13882,9,6,2019,04,25
IndiGo,2019-05-12,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218,12,5,2019,23,30
IndiGo,2019-03-01,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302,1,3,2019,21,35


In [12]:
train_data.drop(['Date_of_Journey', 'Arrival_Time'], axis=1, inplace=True)


In [13]:
train_data.head()

,Source,Destination,Route,Dep_Time,Duration,Total_Stops,Additional_Info,Price,Day_of_Journey,Month_of_Journey,Year_of_Journey,Arrival_hour,Arrival_min
Airline,,,,,,,,,,,,,
IndiGo,Banglore,New Delhi,BLR ? DEL,22:20,2h 50m,non-stop,No info,3897,24,3,2019,01,10
Air India,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,7h 25m,2 stops,No info,7662,1,5,2019,13,15
Jet Airways,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,19h,2 stops,No info,13882,9,6,2019,04,25
IndiGo,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,5h 25m,1 stop,No info,6218,12,5,2019,23,30
IndiGo,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,4h 45m,1 stop,No info,13302,1,3,2019,21,35


In [14]:
train_data['Dep_hour']=train_data['Dep_Time'].str.split(':').str[0]
train_data['Dep_min']=train_data['Dep_Time'].str.split(':').str[1]

train_data['Duration']

Airline
IndiGo         2h 50m
Air India      7h 25m
Jet Airways       19h
IndiGo         5h 25m
IndiGo         4h 45m
                ...  
Air Asia       2h 30m
Air India      2h 35m
Jet Airways        3h
Vistara        2h 40m
Air India      8h 20m
Name: Duration, Length: 10683, dtype: object

In [15]:
train_data.drop(['Dep_Time'],axis=1,inplace=True)

In [16]:
train_data['Dep_hour']=train_data['Dep_hour'].astype(int)
train_data['Dep_min']=train_data['Dep_min'].astype(int)

In [17]:
train_data.head()

,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Day_of_Journey,Month_of_Journey,Year_of_Journey,Arrival_hour,Arrival_min,Dep_hour,Dep_min
Airline,,,,,,,,,,,,,,
IndiGo,Banglore,New Delhi,BLR ? DEL,2h 50m,non-stop,No info,3897,24,3,2019,01,10,22,20
Air India,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,7h 25m,2 stops,No info,7662,1,5,2019,13,15,5,50
Jet Airways,Delhi,Cochin,DEL ? LKO ? BOM ? COK,19h,2 stops,No info,13882,9,6,2019,04,25,9,25
IndiGo,Kolkata,Banglore,CCU ? NAG ? BLR,5h 25m,1 stop,No info,6218,12,5,2019,23,30,18,5
IndiGo,Banglore,New Delhi,BLR ? NAG ? DEL,4h 45m,1 stop,No info,13302,1,3,2019,21,35,16,50


In [19]:
train_data['Total_Stops']=train_data['Total_Stops'].map({'non-stop':0 , '1 stop':1 ,'2 stops': 2, '3 stops':3 , np.nan:1 })

In [20]:
train_data.drop(['Route'],axis=1,inplace=True)

In [21]:
train_data.head()

,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Day_of_Journey,Month_of_Journey,Year_of_Journey,Arrival_hour,Arrival_min,Dep_hour,Dep_min
Airline,,,,,,,,,,,,,
IndiGo,Banglore,New Delhi,2h 50m,0.0,No info,3897,24,3,2019,01,10,22,20
Air India,Kolkata,Banglore,7h 25m,2.0,No info,7662,1,5,2019,13,15,5,50
Jet Airways,Delhi,Cochin,19h,2.0,No info,13882,9,6,2019,04,25,9,25
IndiGo,Kolkata,Banglore,5h 25m,1.0,No info,6218,12,5,2019,23,30,18,5
IndiGo,Banglore,New Delhi,4h 45m,1.0,No info,13302,1,3,2019,21,35,16,50
